In [1]:
import torch
import torchvision
from torch import nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data.dataloader import DataLoader

In [2]:
tranform = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dataset = torchvision.datasets.ImageFolder(root='../input/fruits/fruits-360_dataset/fruits-360/Training', transform=tranform)
test_dataset = torchvision.datasets.ImageFolder(root='../input/fruits/fruits-360_dataset/fruits-360/Test', transform=tranform)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print('Train DataSet len:',len(train_dataset))
print('Train DataSet Classes:',len(train_dataset.classes))
print('Test DataSet len:',len(test_dataset))
print('Test DataSet Classes:',len(test_dataset.classes))

Train DataSet len: 67692
Train DataSet Classes: 131
Test DataSet len: 22688
Test DataSet Classes: 131


In [3]:
class Custom_VGG16(nn.Module):
    def __init__(self):
        super(Custom_VGG16, self).__init__()
        self.ConvSet_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.ConvSet_2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.ConvSet_3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.ConvSet_4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.ConvSet_5 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.FC_Layers = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 131),
        )
        
    def forward(self, x):
        out = self.ConvSet_1(x)
        out = self.ConvSet_2(out)
        out = self.ConvSet_3(out)
        out = self.ConvSet_4(out)
        out = self.ConvSet_5(out)
        out = out.reshape(out.shape[0], -1)
        out = self.FC_Layers(out)
        return out

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Custom_VGG16()
model = model.to(device=device)
print(model)
print('Device: ',device)

Custom_VGG16(
  (ConvSet_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ConvSet_2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ConvSet_3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [5]:
learning_rate = 1e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

In [6]:
num_epochs = 30
for epoch in range(num_epochs):
    loss_ep = 0
    for images, labels in train_dataloader:
        images = images.to(device=device)
        labels = labels.to(device=device) 
        optimizer.zero_grad()
        scores = model(images)
        loss = criterion(scores,labels)
        loss.backward()
        optimizer.step()
        loss_ep += loss.item()
    print ('Epoch [{}/{}] :::: Loss: {:.4f}'.format(epoch+1, num_epochs, loss_ep/len(train_dataloader)))

Epoch [1/30] :::: Loss: 1.8050
Epoch [2/30] :::: Loss: 0.1387
Epoch [3/30] :::: Loss: 0.0631
Epoch [4/30] :::: Loss: 0.0428
Epoch [5/30] :::: Loss: 0.0319
Epoch [6/30] :::: Loss: 0.0302
Epoch [7/30] :::: Loss: 0.0191
Epoch [8/30] :::: Loss: 0.0201
Epoch [9/30] :::: Loss: 0.0165
Epoch [10/30] :::: Loss: 0.0125
Epoch [11/30] :::: Loss: 0.0147
Epoch [12/30] :::: Loss: 0.0120
Epoch [13/30] :::: Loss: 0.0090
Epoch [14/30] :::: Loss: 0.0108
Epoch [15/30] :::: Loss: 0.0076
Epoch [16/30] :::: Loss: 0.0087
Epoch [17/30] :::: Loss: 0.0058
Epoch [18/30] :::: Loss: 0.0068
Epoch [19/30] :::: Loss: 0.0074
Epoch [20/30] :::: Loss: 0.0050
Epoch [21/30] :::: Loss: 0.0066
Epoch [22/30] :::: Loss: 0.0044
Epoch [23/30] :::: Loss: 0.0058
Epoch [24/30] :::: Loss: 0.0042
Epoch [25/30] :::: Loss: 0.0030
Epoch [26/30] :::: Loss: 0.0048
Epoch [27/30] :::: Loss: 0.0037
Epoch [28/30] :::: Loss: 0.0051
Epoch [29/30] :::: Loss: 0.0033
Epoch [30/30] :::: Loss: 0.0033


In [7]:
torch.save(model.state_dict(), "Custom_VGG16.pt")

In [8]:
model = Custom_VGG16()
model.load_state_dict(torch.load("./Custom_VGG16.pt"))
model = model.to(device=device)
model.eval()

Custom_VGG16(
  (ConvSet_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ConvSet_2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ConvSet_3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [9]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device=device)
        labels = labels.to(device=device) 
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the Model on the {} Test Images: {}%'.format(len(test_dataset), 100*correct /total))

Accuracy of the Model on the 22688 Test Images: 93.41061354019746%
